<a href="https://colab.research.google.com/github/Zirescu/self-join-examination/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# An examination of LEFT JOIN on a self-joining dataset

The following data and task is from the PostgreSQL Exercises created by Alisdair Owens.
More specifically: https://pgexercises.com/questions/joins/self2.html

The dataset was copied from [Github](https://github.com/AlisdairO/pgexercises/blob/master/database/clubdata-members.sql), modified slightly and then dumped into ChatGPT so that I could just get a CSV of the original to import into Datacamp. 

When doing the above exercise I was not getting the correct results with my solution. 

Once I saw the solution provided it became a _little_ more clear on what I had done wrong. I've replicated the exercise here so that I could delve into it further and maybe help others with Self joining tables and the LEFT JOIN clause.

# Our membership dataset
- Recommendedby points to an existing club member who referred the current member, if they weren't recommended by an existing member the value is null.


In [6]:
!pip install --quiet duckdb
!pip install --quiet jupysql 
!pip install --quiet duckdb-engine
!pip install --quiet pandas
import duckdb
import pandas as pd
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:///:memory:

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [3]:
members = pd.read_csv('https://raw.githubusercontent.com/Zirescu/self-join-examination/main/clubdata-members.csv?token=GHSAT0AAAAAACBUG56M5SXOAVVHBXHJZSQYZDNECCA')
members = members.replace('\\N', pd.NA)
members.head(10)

,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,<NA>,2012-07-02 12:02:05
1,2,Smith,Tracy,"8 Bloomsbury Close, New York",4321,555-555-5555,<NA>,2012-07-02 12:08:23
2,3,Rownam,Tim,"23 Highway Way, Boston",23423,(844) 693-0723,<NA>,2012-07-03 09:32:15
3,4,Joplette,Janice,"20 Crossing Road, New York",234,(833) 942-4710,1,2012-07-03 10:25:05
4,5,Butters,Gerald,"1065 Huntingdon Avenue, Boston",56754,(844) 078-4130,1,2012-07-09 10:44:09
5,6,Tracy,Burton,"3 Tunisia Drive, Boston",45678,(822) 354-9973,<NA>,2012-07-15 08:52:55
6,7,Dare,Nancy,"6 Hunting Lodge Way, Boston",10383,(833) 776-4001,4,2012-07-25 08:59:12
7,8,Boothe,Tim,"3 Bloomsbury Close, Reading, 00234",234,(811) 433-2547,3,2012-07-25 16:02:35
8,9,Stibbons,Ponder,"5 Dragons Way, Winchester",87630,(833) 160-3900,6,2012-07-25 17:09:05
9,10,Owen,Charles,"52 Cheshire Grove, Winchester, 28563",28563,(855) 542-5251,1,2012-08-03 19:42:37


**Some observations from the above:**
- Member 1 (Darren Smith) was not referred by any member of the club.
- Member 2 (Tracy Smith) and 3 (Tim Rownam) were also not referred by any member of the club. 
- Member 4 (Janice Joplette) was referred to the club by member 1 (Darren Smith)
- Member 5 (Gerrald Butters) was referred to the club by member 1 (Darren Smith)



**Our task:**
- Generate a list of members (first `firstname` and last name `surname`) and the names of the members that recommended them to the club. 
- Order by `surname`, `firstname`.

**Let's break this down:**
1. We have a total of 30 members so our expected output needs to be 30 as well. 
2. As we don't have a secondary table that references the names of the individuals that recommended the member we will need to join the members table on itself. 
3. We will need to perform a `LEFT JOIN` so that we keep the original member list and supplement it by returning the names of the individuals that provided the recommendation. 
4. Therefore, the left-hand side of the join **must** be the member id `memid` from the right-most table that points to the `recommendedby` in the left-hand table.

**Let's simplify the data down to focus on just the values we care about.** 

In [8]:
%%sql -- A trimmed down view of our data.
SELECT 
  memid,
  surname,
  firstname,
  recommendedby
FROM members;

,memid,surname,firstname,recommendedby
0,1,Smith,Darren,None
1,2,Smith,Tracy,None
2,3,Rownam,Tim,None
3,4,Joplette,Janice,1
4,5,Butters,Gerald,1
5,6,Tracy,Burton,None
6,7,Dare,Nancy,4
7,8,Boothe,Tim,3
8,9,Stibbons,Ponder,6
9,10,Owen,Charles,1


**PostgreSQL Definitions for LEFT JOIN**

[LEFT JOIN DEFINITION](https://www.postgresql.org/docs/current/queries-table-expressions.html)
- First, an inner join is performed. 
- Then, for each row in T1 that does not satisfy the join condition with any row in T2, a joined row is added with null values in columns of T2. 
- Thus, the joined table always has at least one row for each row in T1.

[LEFT OUTER JOIN](https://www.postgresql.org/docs/15/sql-select.html) 
- returns all rows in the qualified Cartesian product (i.e., all combined rows that pass its join condition), plus one copy of each row in the left-hand table for which there was no right-hand row that passed the join condition. 
- This left-hand row is extended to the full width of the joined table by inserting null values for the right-hand columns. Note that only the JOIN clause's own condition is considered while deciding which rows have matches. 
- Outer conditions are applied afterwards.

A simplified definition from the Datacamp cheat sheet: 
- A left join keeps all of the original records in the left table and returns missing values for any columns from the right table where the joining field did not find a match.

**Let's write our query such that we can examine it for correctness.**

In [9]:
%%sql SELECT 
 l.memid as memid,  -- left-table memid value
 l.surname as mem_sname,  -- left-table lastname value
 l.firstname as mem_fname, -- left-table firstname value
 l.recommendedby as l_recommendedby, -- left-table recommendedby value
 r.memid as r_memid, -- our right-table memid value which should be the same as the l_recommendedby value
 r.surname as rec_sname, -- the last name of the person that recommended the member
 r.firstname as rec_fname, -- the first name of the person that recommended the member
 r.recommendedby as r_reommendedby, -- the member id of any person that was recommended by the recommendy
FROM members as l
LEFT JOIN members as r
ON r.memid = l.recommendedby -- where member ID is the recommendedby list ID
ORDER BY mem_sname, mem_fname; -- order by surname, firstname

,memid,mem_sname,mem_fname,l_recommendedby,r_memid,rec_sname,rec_fname,r_reommendedby
0,15,Bader,Florence,9,9.0,Stibbons,Ponder,6
1,12,Baker,Anne,9,9.0,Stibbons,Ponder,6
2,16,Baker,Timothy,13,13.0,Farrell,Jemima,None
3,8,Boothe,Tim,3,3.0,Rownam,Tim,None
4,5,Butters,Gerald,1,1.0,Smith,Darren,None
5,22,Coplin,Joan,16,16.0,Baker,Timothy,13
6,36,Crumpet,Erica,2,2.0,Smith,Tracy,None
7,7,Dare,Nancy,4,4.0,Joplette,Janice,1
8,28,Farrell,David,None,NaN,None,None,None
9,13,Farrell,Jemima,None,NaN,None,None,None


**Let's examine the above result for correctness:**
1. Our output is 30 rows as expected. 
2. The `l_recommendedby` value is equal to `r_memid` so we know that we've grabbed the correct member info. 
3. We can manually confirm that the names match as well. 

**Now that we've confirmed our results we can simplify the query down to remove the extraneous info.**

In [10]:
%%sql -- Simplified query to display only the columns we need
SELECT 
 l.surname as mem_sname,  -- left-table lastname value
 l.firstname as mem_fname, -- left-table firstname value
 r.surname as rec_sname, -- the last name of the person that recommended the member
 r.firstname as rec_fname -- the first name of the person that recommended the member
FROM members as l
LEFT JOIN members as r
ON r.memid = l.recommendedby -- where member ID is the recommendeby list
ORDER BY mem_sname, mem_fname; -- order by surname, firstname

,mem_sname,mem_fname,rec_sname,rec_fname
0,Bader,Florence,Stibbons,Ponder
1,Baker,Anne,Stibbons,Ponder
2,Baker,Timothy,Farrell,Jemima
3,Boothe,Tim,Rownam,Tim
4,Butters,Gerald,Smith,Darren
5,Coplin,Joan,Baker,Timothy
6,Crumpet,Erica,Smith,Tracy
7,Dare,Nancy,Joplette,Janice
8,Farrell,David,None,None
9,Farrell,Jemima,None,None


___
If you've made it this far thanks for reading! The following is the rabbit hole I went down when I initially got the solution wrong. The error that I had made was related to order of fields in the ON condition. In the majority of the ON conditions I've seen and used, the order hasn't _seemed_ to matter. 

Below is what happens if the fields of the ON condition are swapped. 

In [12]:
%%sql -- NOT the desired output 
/* When the l.memid is on the left-hand side of the equal operator we will return the names of the members that they recommended and not the names of the who they were recommended by. 
*/ 
SELECT 
 l.memid as l_memid,
 l.surname as mem_sname, 
 l.firstname as mem_fname, 
 l.recommendedby as l_recommendedby,
 r.recommendedby as r_recommendedby,
 r.memid as r_memid,
 r.surname as rec_sname,
 r.firstname as rec_fname
FROM members as l
LEFT JOIN members as r
ON l.memid = r.recommendedby -- think of this as give me the records where the left most value appears in the right most value 
ORDER BY mem_sname, mem_fname; 

,l_memid,mem_sname,mem_fname,l_recommendedby,r_recommendedby,r_memid,rec_sname,rec_fname
0,15,Bader,Florence,9,15,24.0,Sarwin,Ramnaresh
1,12,Baker,Anne,9,None,NaN,None,None
2,16,Baker,Timothy,13,16,22.0,Coplin,Joan
3,8,Boothe,Tim,3,None,NaN,None,None
4,5,Butters,Gerald,1,5,20.0,Genting,Matthew
5,22,Coplin,Joan,16,None,NaN,None,None
6,36,Crumpet,Erica,2,None,NaN,None,None
7,7,Dare,Nancy,4,None,NaN,None,None
8,28,Farrell,David,None,None,NaN,None,None
9,13,Farrell,Jemima,None,13,17.0,Pinker,David


**Analyzing the above query:**
1. We end up with 39 rows instead of 30 rows! We'll dig into this further, but we ended up wtih 22 records which matched and 17 that didn't. 
2. As we can see from the above instead of returning the names of the members that were 'recommeded by' we actually returned back the list of members that the current member recommended. Without the additional fields this is more difficult to validate. 

**Example:**
- Member 15 (Florence Bader) was recommended by Member 9 (Ponder Stibbins), but we show Member 24 (Ramnaresh Sarwin) which was who Florence Bader recommended to the club. 

When I originally saw I was wrong and then looked at the solution it was more evident that the order of the ON condition mattered a lot in this case. I switched them around and voila, the correct solution. 

But now my brain was wondering: _"Why was there 39 rows?"_. To figure this out I went back to the documentation on LEFT joins, which honestly didn't make it that much clearer. I could've stopped there but I _needed_ to know why and went looking for the reasons online. I didn't find anything that really helped so I went back to the definition and started to experiment to figure out what I think is going on behind the scenes. 

What follows is that thought experiment...

# An examination of a LEFT JOIN 
We can breakdown the steps of the LEFT JOIN with the following three queries using our prior dataset.

## First-part of Left JOIN
- First, an inner join is performed.


In [16]:
%%sql first_part < -- Inner JOIN query and results which generates 22 matching records
-- This will only produce records where the member has been recommended by another member.
SELECT 
  m1.memid as memid,
  m1.surname as surname,
  m1.firstname as firstname,
  m1.recommendedby as recommededby
FROM members m1
INNER JOIN members m2
ON m1.memid = m2.recommendedby
ORDER BY m1.surname, m1.firstname;

(duckdb.ParserException) Parser Error: syntax error at or near "first_part"
LINE 1: first_part <
        ^
[SQL: first_part <
-- This will only produce records where the member has been recommended by another member.
SELECT 
  m1.memid as memid,
  m1.surname as surname,
  m1.firstname as firstname,
  m1.recommendedby as recommededby
FROM members m1
INNER JOIN members m2
ON m1.memid = m2.recommendedby
ORDER BY m1.surname, m1.firstname;]
(Background on this error at: https://sqlalche.me/e/20/f405)


<h2 id="heading2"> Second-part of Left JOIN operation</h2>  

- Then, for each row in T1 that does not satisfy the join condition with any row in T2, a joined row is added with null values in columns of T2.Thus, the joined table always has at least one row for each row in T1.  

The above sounded like something I had read before in the Datacamp Cheat sheet which was an "ANTI JOIN".
- The anti join chooses records in the first table where a condition is `NOT` met in the second table. It makes use of a `WHERE` clause to use exclude values from the second table.

Breaking that down further, if we take the results of the `first_part` (above) and do an ANTI JOIN that should in theory give us a list of all the records that did not match the INNER JOIN. 

**Anti Join Query**

In [14]:
%%sql -- Second-part of the left join operation
-- To obtain the other 17 records
SELECT 
  memid, 
  surname, 
  firstname,
  recommendedby
FROM members
WHERE memid NOT IN (
	SELECT memid 
	FROM first_part);

(duckdb.CatalogException) Catalog Error: Table with name first_part does not exist!
Did you mean "pg_am"?
[SQL: -- To obtain the other 17 records
SELECT 
  memid, 
  surname, 
  firstname,
  recommendedby
FROM members
WHERE memid NOT IN (
	SELECT memid 
	FROM first_part);]
(Background on this error at: https://sqlalche.me/e/20/f405)


**About the Anti Join Query above:**

To find the 17 records that did not match on the `first_part` we need to perform an ANTI JOIN query (essentially a subquery) where we want to find all the records that did not appear in the INNER JOIN query. 

To do that we query our orginal data set but use a WHERE with a subquery to look-up all the records where the mem_id did **not** appear in the results of the INNER JOIN (`first_part`). 

## Third-part - UNION ALL the two result sets together

Now that we've got the two results sets we just need to UNION them together. We need to use `UNION ALL` so that we maintain any duplicates.

In [ ]:
-- COMBINE the two results set together
SELECT *
FROM first_part
UNION ALL -- We need to keep the duplicates!
SELECT *
FROM second_part
ORDER BY surname, firstname;

,memid,surname,firstname,recommededby
0,15,Bader,Florence,9
1,12,Baker,Anne,9
2,16,Baker,Timothy,13
3,8,Boothe,Tim,3
4,5,Butters,Gerald,1
5,22,Coplin,Joan,16
6,36,Crumpet,Erica,2
7,7,Dare,Nancy,4
8,28,Farrell,David,NaN
9,13,Farrell,Jemima,NaN


There we have it, all 39 records which match up with the results from the LEFT JOIN. 

If we hadn't included the ALL with UNION we get only 30 rows as shown below.

In [ ]:
-- COMBINE the two results set together without the ALL
SELECT *
FROM first_part
UNION  -- Without ALL!
SELECT *
FROM second_part
ORDER BY surname, firstname;

,memid,surname,firstname,recommededby
0,15,Bader,Florence,9
1,12,Baker,Anne,9
2,16,Baker,Timothy,13
3,8,Boothe,Tim,3
4,5,Butters,Gerald,1
5,22,Coplin,Joan,16
6,36,Crumpet,Erica,2
7,7,Dare,Nancy,4
8,28,Farrell,David,NaN
9,13,Farrell,Jemima,NaN


**If you've made it this far I applaud you and I hope this has helped shed some light on LEFT JOIN and the order of the ON condition fields.**

___
# An example for how to find out which members have been recommened by another member. 

In [ ]:
-- This will only produce records where the member has been recommended by another member
SELECT m1.memid as m1_memid,
m1.surname as mem_surname,
m1.firstname as mem_firstname,
m1.recommendedby,
m2.memid as m2_memid,
m2.surname as rec_surname,
m2.firstname as rec_firstname
FROM members m1, members m2
WHERE m2.memid = m1.recommendedby
ORDER BY m1.surname, m1.firstname

,m1_memid,mem_surname,mem_firstname,recommendedby,m2_memid,rec_surname,rec_firstname
0,15,Bader,Florence,9,9,Stibbons,Ponder
1,12,Baker,Anne,9,9,Stibbons,Ponder
2,16,Baker,Timothy,13,13,Farrell,Jemima
3,8,Boothe,Tim,3,3,Rownam,Tim
4,5,Butters,Gerald,1,1,Smith,Darren
5,22,Coplin,Joan,16,16,Baker,Timothy
6,36,Crumpet,Erica,2,2,Smith,Tracy
7,7,Dare,Nancy,4,4,Joplette,Janice
8,20,Genting,Matthew,5,5,Butters,Gerald
9,35,Hunt,John,30,30,Purview,Millicent


The above and the below are equivalent.

In [ ]:
-- This will only produce records where the member has been recommended by another member
SELECT m1.memid as m1_memid,
m1.surname as mem_surname,
m1.firstname as mem_firstname,
m1.recommendedby,
m2.memid as m2_memid,
m2.surname as rec_surname,
m2.firstname as rec_firstname
FROM members m1
INNER JOIN members m2
ON m2.memid = m1.recommendedby
ORDER BY m1.surname, m1.firstname

,m1_memid,mem_surname,mem_firstname,recommendedby,m2_memid,rec_surname,rec_firstname
0,15,Bader,Florence,9,9,Stibbons,Ponder
1,12,Baker,Anne,9,9,Stibbons,Ponder
2,16,Baker,Timothy,13,13,Farrell,Jemima
3,8,Boothe,Tim,3,3,Rownam,Tim
4,5,Butters,Gerald,1,1,Smith,Darren
5,22,Coplin,Joan,16,16,Baker,Timothy
6,36,Crumpet,Erica,2,2,Smith,Tracy
7,7,Dare,Nancy,4,4,Joplette,Janice
8,20,Genting,Matthew,5,5,Butters,Gerald
9,35,Hunt,John,30,30,Purview,Millicent
